In [60]:
import pandas as pd
import numpy as np
from typing import Optional
from datetime import date
import math
from dataclasses import asdict, dataclass

In [95]:
df = pd.read_excel('1.ОСНОВНОЙ Факт объемы_м3_2017_2022_по дням1.xlsx', header=1, sheet_name=2)
df

,склад,МСК,СЗ,Урал,Хранение,Unnamed: 5,МСК.1,СЗ.1,Урал.1,Обработка,...,МСК.4,СЗ.4,Урал.4,всего Хранение,Unnamed: 23,Unnamed: 24,МСК.5,СЗ.5,Урал.5,всего Обработка
0,2019-01-01,4507.730000,1549.08560,1932.45,7989.265600,NaN,9.480000,0.00,0.00,9.480000,...,6500,2200,2000,10700,NaN,январь,3000.0,800.0,1000.0,4800.0
1,2019-01-02,4507.730000,1549.08560,1932.45,7989.265600,NaN,0.000000,0.00,0.00,0.000000,...,6500,2200,2000,10700,NaN,февраль,3000.0,800.0,1000.0,4800.0
2,2019-01-03,4507.730000,1549.08560,1932.45,7989.265600,NaN,0.000000,0.00,0.00,0.000000,...,6500,2200,2000,10700,NaN,март,3000.0,800.0,1000.0,4800.0
3,2019-01-04,4507.730000,1549.08560,1932.45,7989.265600,NaN,0.000000,0.00,0.00,0.000000,...,6500,2200,2000,10700,NaN,апрель,2500.0,800.0,750.0,4050.0
4,2019-01-05,4507.730000,1549.08560,1932.45,7989.265600,NaN,0.000000,0.00,0.00,0.000000,...,6500,2200,2000,10700,NaN,май,2500.0,900.0,900.0,4300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2019-12-27,4697.124465,4061.49425,2100.83,10859.448715,NaN,58.684113,94.70,61.16,214.544113,...,6000,6000,1800,13800,NaN,NaN,NaN,NaN,NaN,NaN
361,2019-12-28,4681.662810,4225.29425,2091.78,10998.737060,NaN,0.000000,198.40,0.00,198.400000,...,6000,6000,1800,13800,NaN,NaN,NaN,NaN,NaN,NaN
362,2019-12-29,4681.662810,4225.29425,2091.78,10998.737060,NaN,0.000000,0.00,0.00,0.000000,...,6000,6000,1800,13800,NaN,NaN,NaN,NaN,NaN,NaN
363,2019-12-30,4694.885474,4702.21000,2113.95,11511.045474,NaN,64.451454,63.73,40.54,168.721454,...,6000,6000,1800,13800,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
@dataclass
class Row:
    date: date
    keeping_actual: float
    keeping_reserved: float
    processing_actual: float
    receiving_actual: float
    shipment_actual: float  
    processing_reserved: float
        
month_ru_to_num = {"январь": 1, "февраль": 2, "март": 3, "апрель": 4, "май": 5, "июнь": 6, "июль": 7, "август": 8, "сентябрь": 9, "октябрь": 10, "ноябрь": 11, "декабрь": 12}

In [99]:
### SHEET 1
df = pd.read_excel('1.ОСНОВНОЙ Факт объемы_м3_2017_2022_по дням1.xlsx', header=1, sheet_name=1)
df
warehouses = {"msk": [], "northwest": [], "ural": []}
def process_sheet():
    processing_reserved_month = {"msk": {}, "northwest": {}, "ural": {}}
    for idx, row in df.iterrows():
        if idx == 12:
            break

        month = month_ru_to_num[row['Unnamed: 24'].lower()]
        processing_reserved_month['msk'][month] = row['МСК.5']
        processing_reserved_month['northwest'][month] = row['СЗ.5']
        processing_reserved_month['ural'][month] = row['Урал.5']

    for idx, row in df.iterrows():
        cur_date = row['склад']
        row_ = Row(date=cur_date, 
                   keeping_actual=row['МСК'], 
                   processing_actual=row['МСК.1'],
                   receiving_actual=row['МСК.2'],
                   shipment_actual=row['МСК.3'],
                   keeping_reserved=row['МСК.4'], 
                   processing_reserved=processing_reserved_month['msk'][cur_date.month])
        warehouses['msk'].append(row_)

        row_ = Row(date=cur_date, 
               keeping_actual=row['СЗ'], 
               processing_actual=row['СЗ.1'],
               receiving_actual=row['СЗ.2'],
               shipment_actual=row['СЗ.3'],
               keeping_reserved=row['СЗ.4'], 
               processing_reserved=processing_reserved_month['northwest'][cur_date.month])
        warehouses['northwest'].append(row_)

        row_ = Row(date=cur_date, 
           keeping_actual=row['Урал'], 
           processing_actual=row['Урал.1'],
           receiving_actual=row['Урал.2'],
           shipment_actual=row['Урал.3'],
           keeping_reserved=row['Урал.4'], 
           processing_reserved=processing_reserved_month['ural'][cur_date.month])
        warehouses['ural'].append(row_)
    
mow_df = pd.DataFrame((asdict(row) for row in warehouses['msk'])).fillna(0.0)
ural_df = pd.DataFrame((asdict(row) for row in warehouses['ural'])).fillna(0.0)
northwest_df = pd.DataFrame((asdict(row) for row in warehouses['northwest'])).fillna(0.0)
mow_df

AttributeError: 'float' object has no attribute 'lower'

True